## Working on Time Series with Pandas


* Forecasting: Principles and Practice (3rd ed) [link](https://otexts.com/fpp3/)
* Pandas documentation for time series(https://pandas.pydata.org/docs/user_guide/timeseries.html)

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
plt.style.use('dark_background')
%matplotlib inline

In [ ]:
## Generate three timestamps starting from "2023-01-01" with frequency of "1 hr"

ts_index = pd.date_range('2023-01-01', periods = 3, freq = 'H')
ts_index

Some of the commonly used `freq` tags

| Date Offset | Frequency String | Description        |
|-------------|------------------|--------------------|
| MonthEnd    | 'M'              | calendar month end |
| Day         | 'D'              | one absolute day   |
| Hour        | 'H'              | one hour           |
| Minute      | 'T' or 'min'     | one minute         |
| Second      | 'S'              | one second         |

*Check this for other supported "freq" tags:
https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects*



**Manipulating and converting date times with timezone information**

In [ ]:
## Generate timestamps specific to timezones

# Time zone UTC
print(ts_index.tz_localize('UTC'))

# Time zone Asia/kokata
print(ts_index.tz_localize("Asia/kolkata"))

____
**Task - 1**
____

Your manager has given you a climate timeseries dataset with 1000 rows and asked you to analyse it but the dataset **does not have a timestamp column**. You are informed that each row of this dataset represents outputs from different sensors.

Your manager added that the observations were made starting from "*12th jan 2020*" and is **daily data**, meaning one obeservation recorded per day starting from `2020-01-12`.

* Create a datetime index in pandas starting from `12th jan 2020` with `1000` observatiions.

* Add timestamps as index to the dataset.



In [ ]:
# Read climate data set
climate_data = pd.read_csv('https://tinyurl.com/mpbudws')
print(climate_data.head(5))

In [ ]:
# Generate timestamp values
start_date = '2020-01-12'
data_length = climate_data.shape[0]
timestamps = pd.date_range(start = start_date, periods = data_length, freq = 'D')

# Add timestamp column to the climate_data
climate_data['timestamps'] = timestamps
climate_data = climate_data.set_index('timestamps')
print(climate_data.head(5))

____

**Resampling a time series**

In [ ]:
## Genrate hourly data with random values for 10 periods
idx = pd.date_range('2010-01-01', periods = 10, freq = '1H')
df = pd.DataFrame({'Values': np.random.rand(10)}, index = idx)
df.head()

To perform resampling, the daraframe index has to be in pandas DateTime format and we need to specify the aggregation fuction like `mean()`, `min()`, `max()`, etc. Refer to official documentation of pandas resample for more information. [pandas.DataFrame.resample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html)

In [ ]:
## Resample (in this case, downsample) the data to z frequency of "2 hours"
df.resample('2H').mean()

____
**Task - 2**
____

From your initial analysis of the climate data, you came to know that there is only a slight variation in the data on a daily basis. It is redundant to have too many values with no extra information and might cause computational overhead. So, you decide to convert the daily data to weekly data. In other words, you want to resample the climate time series from `1 day` to  `1 week`. The resampled values must be the maximum observation over the respective weeks.


In [ ]:
## Resample the climate time series from 1 day to 1 week
## with values as maximum over the week
climate_data_resampled = climate_data.resample('W').max()
climate_data_resampled.head()

Plot the climate time series data

In [ ]:
## Plot climate timeseries data
colors = cm.rainbow(np.linspace(0, 1, 6))
fig, ax = plt.subplots(2, 3, figsize = (14, 6), tight_layout = True)
df_plot = climate_data_resampled
for i in range(2):
  for j in range(3):
    col = df_plot.columns[i+2*j]
    ax[i,j].plot(df_plot.index, df_plot[col], color = colors[i+2*j])
    ax[i, j].set_xlabel('Timestamp', fontsize = 8)
    ax[i, j].set_ylabel('Values', fontsize = 8)
    ax[i, j].set_title(col, fontsize = 12)
    ax[i, j].xaxis.set_major_locator(mdates.MonthLocator(bymonth = range(1, 12, 3)))
    ax[i, j].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
    ax[i, j].tick_params(axis = 'x', rotation = 90);

**Performing date and time arithmetic with absolute or relative time increments**

In [ ]:
date1 = pd.Timestamp('2018-01-05')

# add 2 days, 5 hours and 10 minutes to 1st date
date2 = date1 + pd.Timedelta('2D 5H 10min')

date2

____

**Task - 3**
____

Assume that the data you have was collected in India. You want to store this data in an internal database, and your database engineer tells you that their team can only work with `timestamps without time zone` (24  hour clock UTC). Achieve this by subtracting 5 hours and 30 minutes from all the values in the timestamp index.




In [ ]:
climate_data.head()

In [ ]:
climate_data.index = climate_data.index - pd.Timedelta('5H 30min')
climate_data.head()

____

### **Time Series Data Wrangling and Visualization**



Vizualising a subset of data using groupby and pivot. The data set consists of timestamps column `Date` (frequency is 1 day), `store` (store id), `product` (product id)  and `number_sold` (number of a product sold by a store).

In [ ]:
## Data source: https://www.kaggle.com/datasets/samuelcortinhas/time-series-practice-dataset
sales_data = pd.read_csv('https://tinyurl.com/mr2rv4yh')
sales_data.head()

____

**Task - 3**
____


Convert 'date' column type to pandas.datetime and set date as index of the dataframe

In [ ]:
# Convert 'Date' column type to pandas.datetime
sales_data['Date'] = pd.to_datetime(sales_data['Date'])

# Set Date as index of the dataframe
sales_data.set_index('Date', inplace = True)

sales_data.head()

In [ ]:
## Resample and forward fill null values
grouping_cols = ['store', 'product']

def resample_fn(data):
  """Resamples the time series to frequency
  equal to 1 Week, using average of values
  over the week and forward fill the nan values
  if present.
  """
  return data.resample('W').mean().fillna(method = 'ffill')

## Resample the data belonging to each product of each store seperately
## and group the data by 'store' and 'product' by applying the fuction above
sales_data_processed = sales_data.groupby(grouping_cols, as_index = False).apply(resample_fn)
sales_data_processed.head()

In [ ]:
## Remove MultiIndex, only Date is required as Index
sales_data_processed = sales_data_processed.reset_index(level = 0, drop = True)
sales_data_processed.head()

**Plot the sales of all the products from store id 0**

In [ ]:
## Filter the data for store id equal to 0
store_id_0 = sales_data_processed[sales_data_processed['store'] == 0]

In [ ]:
## Modify the data such that index represents date and
## columns represent the product id

store_0_pivot = store_id_0.pivot(columns = 'product', values = 'number_sold')
store_0_pivot.head()

In [ ]:
store_0_pivot.plot(figsize = (10, 8))

Analyze how the sales of `product 1` of `store 0` have changed over the years

In [ ]:
# Filter product id equal to 1 from store_0
product_1_s0 = store_id_0[store_id_0['product'] == 1]

In [ ]:
product_1_s0.head()

In [ ]:
# Create a column called year
product_1_s0['Year'] = product_1_s0.index.year

# Create a column called month
product_1_s0['Month'] = product_1_s0.index.month

# create a DataFrame whose index represents months and columns respresent years
result_df = pd.pivot_table(product_1_s0,
                           index = 'Month',
                           columns = 'Year',
                           values = 'number_sold',
                           aggfunc = np.mean
                           )
result_df.head()

In [ ]:
result_df.plot(figsize=(10,8))

**Moving average**

In [ ]:
## Load stock price data
df_stock = pd.read_csv("https://tinyurl.com/3fh8j4x8")
df_stock.head()

In [ ]:
## Convert date to pandas Date Time type, set Date as index
## and resample the data to 1 day
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df_stock.set_index(['Date'], inplace = True)
sampling_rate = df_stock.index[1] - df_stock.index[0]
df_stock.resample(sampling_rate).mean().interpolate(method = 'linear')

In [ ]:
## Extract closing price of the stock and plot it
stock_price = df_stock[['Close']]
print(stock_price.head())
stock_price.plot();

In [ ]:
## Smoothen plot using moving average


**Lag**

In [ ]:
## Compute 4 lags for the closing stock price and add these as
## additional columns to the Dataframe
lag_1 = stock_price.shift(-1)
lag_2 =
lag_3 =
lag4 =

stock_price.loc[:, ['lag_1', 'lag_2', 'lag_3', 'lag_4']] = [lag_1, lag_2, lag_3, lag_4]
stock_price.head(4)

In [ ]:
## Plot lagged timeseries data for the last 10 timestamps


**Autocorrelation**

In [ ]:
## Calculate autocorrelation between timeseries and its
## lagged versions


In [ ]:
## Calculate correlation of closing price with all
## lags
for i in ["lag_1","lag_2","lag_3","lag_4"]:

In [ ]:
## Plot 3 different timeseries: constant, linear, and
## white noise
plt.plot(np.arange(1000),np.repeat(5,1000), label="straight line")
plt.show()
plt.plot(np.arange(1000),np.arange(1000))
plt.show()
plt.plot(np.random.randn(1000))
plt.show()